http://billchambers.me/tutorials/2015/01/14/python-nlp-cheatsheet-nltk-scikit-learn.html

http://glowingpython.blogspot.de/2013/07/combining-scikit-learn-and-ntlk.html

http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html

https://stackoverflow.com/questions/10526579/use-scikit-learn-to-classify-into-multiple-categories

https://github.com/anuraagvak/IRE-PersonalityRecognition-Final/blob/master/ire_report.pdf

https://github.com/Charudatt89/Personality_Recognition/blob/master/22-9-PersonalityRecognition/Report/Report.pdf

http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python

Lexical and syntactic features: http://www.aicbt.com/authorship-attribution/

In [15]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
ran.seed(5125)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline

from pandas_confusion import ConfusionMatrix
from scipy.cluster.vq import whiten

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *

# n.download()

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### Read data and show them

In [6]:
data = pd.read_csv("data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=2)
# data.describe()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [7]:
d = {'n': "no", 'y': "yes"}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

### Split data into train + test


In [8]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66)
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66)
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66)
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66)
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66)
# print(len(train_neu))

### Creates diff. pipelines here!

with stop_words : ‘english’ in CountVecto or T.Vecto accuracy ussualy lower

In [9]:
target_names = ['yes', 'no']

# Support Vector Machines
classifierLinearSVC = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('clf', LinearSVC())]) # OneVsRestClassifier already implemented, docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification

classifierSVC = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('clf', SVC(cache_size=500))]) # "one-against-one", docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification

# Two Naybe Bayes below are also same
classifierOVR_NB = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range = (1,3))),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(MultinomialNB()))]) # OneVsOneClassifier does not play role

classifierNB = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range = (1,3))),
    ('tfidf', TfidfTransformer()),
    ('nb', MultinomialNB())])

classifierBNB = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('bnb', BernoulliNB())])

# Nearest Neighboars
classifier_kNN = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('kNN', KNeighborsClassifier())]) # http://scikit-learn.org/stable/modules/neighbors.html#neighbors

# Random Forest
classifierRF = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('rfc', RandomForestClassifier(n_jobs=-1))])

classifierADC = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('adc', AdaBoostClassifier())])


### cNEU

In [18]:
clas_fit_SVC = classifierSVC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_SVC = classifierSVC.predict(test_neu["cNEU"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_LSVC = classifierLinearSVC.predict(test_neu["cNEU"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_BNB = classifierBNB.predict(test_neu["cNEU"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_NB = classifierNB.predict(test_neu["cNEU"])
scores_NB = cross_validation.cross_val_score(classifierNB, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

clas_fit_OVR_NB = classifierOVR_NB.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_OVR_NB = classifierOVR_NB.predict(test_neu["cNEU"])
scores_OVR_NB = cross_validation.cross_val_score(classifierOVR_NB, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of one-vs-rest NB: %0.3f (+/- %0.2f)" % (scores_OVR_NB.mean(), scores_OVR_NB.std() * 2))


#count = 0
#for item, labels in zip(test_neu["STATUS"], predicted_SVC):
#    count += 1
# print(count) # just for checking the count above +- 6000 and here 3372

Accuracy rbf kernel SVC: 0.625 (+/- 0.00)
Accuracy LinearSVC: 0.579 (+/- 0.02)

Accuracy of bernuli NB: 0.624 (+/- 0.00)
Accuracy NB: 0.625 (+/- 0.00)
Accuracy of one-vs-rest NB: 0.625 (+/- 0.00)


In [19]:
clas_fit_kNN = classifier_kNN.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_kNN = classifier_kNN.predict(test_neu["cNEU"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_ADC = classifierADC.predict(test_neu["cNEU"])
scores_ADC = cross_validation.cross_val_score(classifierADC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_RF = classifierRF.predict(test_neu["cNEU"])
scores_RF = cross_validation.cross_val_score(classifierRF, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy of kNN: 0.409 (+/- 0.05)

Accuracy of AdaBoost: 0.606 (+/- 0.02)
Accuracy of random Forest: 0.593 (+/- 0.02)


#### VotingClassifier cNUE

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [20]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("ovr_nb", clas_fit_OVR_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_OVR_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB","OVR_NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, neu["STATUS"], neu["cNEU"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.625 (+/- 0.00) [SVC]
Accuracy: 0.579 (+/- 0.01) [LSVC]
Accuracy: 0.624 (+/- 0.00) [BNB]
Accuracy: 0.625 (+/- 0.00) [NB]
Accuracy: 0.625 (+/- 0.00) [OVR_NB]
Accuracy: 0.409 (+/- 0.03) [kNN]
Accuracy: 0.596 (+/- 0.02) [Random Forest]
Accuracy: 0.606 (+/- 0.01) [ADC]
Accuracy: 0.625 (+/- 0.00) [Ensemble]


### cAGR


In [ ]:
clas_fit_SVC = classifierSVC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_SVC = classifierSVC.predict(test_agr["cAGR"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_LSVC = classifierLinearSVC.predict(test_agr["cAGR"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_BNB = classifierBNB.predict(test_agr["cAGR"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_NB = classifierNB.predict(test_agr["cAGR"])
scores_NB = cross_validation.cross_val_score(classifierNB, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

clas_fit_OVR_NB = classifierOVR_NB.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_OVR_NB = classifierOVR_NB.predict(test_agr["cAGR"])
scores_OVR_NB = cross_validation.cross_val_score(classifierOVR_NB, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of one-vs-rest NB: %0.3f (+/- %0.2f)" % (scores_OVR_NB.mean(), scores_OVR_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_kNN = classifier_kNN.predict(test_agr["cAGR"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_ADC = classifierADC.predict(test_agr["cAGR"])
scores_ADC = cross_validation.cross_val_score(classifierADC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_RF = classifierRF.predict(test_agr["cAGR"])
scores_RF = cross_validation.cross_val_score(classifierRF, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy rbf kernel SVC: 0.531 (+/- 0.00)
Accuracy LinearSVC: 0.512 (+/- 0.03)

Accuracy of bernuli NB: 0.531 (+/- 0.02)
Accuracy NB: 0.526 (+/- 0.02)
Accuracy of one-vs-rest NB: 0.526 (+/- 0.02)

Accuracy of kNN: 0.483 (+/- 0.01)

Accuracy of AdaBoost: 0.513 (+/- 0.05)
Accuracy of random Forest: 0.502 (+/- 0.01)


#### VotingClassifier cAGR

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [ ]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("ovr_nb", clas_fit_OVR_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_OVR_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB","OVR_NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, agr["STATUS"], agr["cAGR"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.531 (+/- 0.00) [SVC]
Accuracy: 0.512 (+/- 0.02) [LSVC]
Accuracy: 0.531 (+/- 0.01) [BNB]
Accuracy: 0.526 (+/- 0.01) [NB]
Accuracy: 0.526 (+/- 0.01) [OVR_NB]
Accuracy: 0.483 (+/- 0.01) [kNN]
Accuracy: 0.500 (+/- 0.01) [Random Forest]
Accuracy: 0.513 (+/- 0.02) [ADC]

### cOPN

In [ ]:
clas_fit_SVC = classifierSVC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_SVC = classifierSVC.predict(test_opn["cOPN"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_LSVC = classifierLinearSVC.predict(test_opn["cOPN"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_BNB = classifierBNB.predict(test_opn["cOPN"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_NB = classifierNB.predict(test_opn["cOPN"])
scores_NB = cross_validation.cross_val_score(classifierNB, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

clas_fit_OVR_NB = classifierOVR_NB.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_OVR_NB = classifierOVR_NB.predict(test_opn["cOPN"])
scores_OVR_NB = cross_validation.cross_val_score(classifierOVR_NB, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of one-vs-rest NB: %0.3f (+/- %0.2f)" % (scores_OVR_NB.mean(), scores_OVR_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_kNN = classifier_kNN.predict(test_opn["cOPN"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_ADC = classifierADC.predict(test_opn["cOPN"])
scores_ADC = cross_validation.cross_val_score(classifierADC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_RF = classifierRF.predict(test_opn["cOPN"])
scores_RF = cross_validation.cross_val_score(classifierRF, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

#### VotingClassifier cOPN

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [ ]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("ovr_nb", clas_fit_OVR_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_OVR_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB","OVR_NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, opn["STATUS"], opn["cOPN"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

### cCON

In [ ]:
clas_fit_SVC = classifierSVC.fit(train_con["STATUS"], train_con["cCON"])
predicted_SVC = classifierSVC.predict(test_con["cCON"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_con["STATUS"], train_con["cCON"])
predicted_LSVC = classifierLinearSVC.predict(test_con["cCON"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_con["STATUS"], train_con["cCON"])
predicted_BNB = classifierBNB.predict(test_con["cCON"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_con["STATUS"], train_con["cCON"])
predicted_NB = classifierNB.predict(test_con["cCON"])
scores_NB = cross_validation.cross_val_score(classifierNB, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

clas_fit_OVR_NB = classifierOVR_NB.fit(train_con["STATUS"], train_con["cCON"])
predicted_OVR_NB = classifierOVR_NB.predict(test_con["cCON"])
scores_OVR_NB = cross_validation.cross_val_score(classifierOVR_NB, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of one-vs-rest NB: %0.3f (+/- %0.2f)" % (scores_OVR_NB.mean(), scores_OVR_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_con["STATUS"], train_con["cCON"])
predicted_kNN = classifier_kNN.predict(test_con["cCON"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_con["STATUS"], train_con["cCON"])
predicted_ADC = classifierADC.predict(test_con["cCON"])
scores_ADC = cross_validation.cross_val_score(classifierADC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_con["STATUS"], train_con["cCON"])
predicted_RF = classifierRF.predict(test_con["cCON"])
scores_RF = cross_validation.cross_val_score(classifierRF, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

#### VotingClassifier cCON

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [ ]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("ovr_nb", clas_fit_OVR_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_OVR_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB","OVR_NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, con["STATUS"], con["cCON"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

### cEXT

In [ ]:
clas_fit_SVC = classifierSVC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_SVC = classifierSVC.predict(test_ext["cEXT"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_LSVC = classifierLinearSVC.predict(test_ext["cEXT"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_BNB = classifierBNB.predict(test_ext["cEXT"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_NB = classifierNB.predict(test_ext["cEXT"])
scores_NB = cross_validation.cross_val_score(classifierNB, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

clas_fit_OVR_NB = classifierOVR_NB.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_OVR_NB = classifierOVR_NB.predict(test_ext["cEXT"])
scores_OVR_NB = cross_validation.cross_val_score(classifierOVR_NB, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of one-vs-rest NB: %0.3f (+/- %0.2f)" % (scores_OVR_NB.mean(), scores_OVR_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_kNN = classifier_kNN.predict(test_ext["cEXT"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_ADC = classifierADC.predict(test_ext["cEXT"])
scores_ADC = cross_validation.cross_val_score(classifierADC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_RF = classifierRF.predict(test_ext["cEXT"])
scores_RF = cross_validation.cross_val_score(classifierRF, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

#### VotingClassifier cEXT

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [ ]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("ovr_nb", clas_fit_OVR_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_OVR_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB","OVR_NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, ext["STATUS"], ext["cEXT"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

### Lexical and other features

In [21]:


def LexicalFeatures(chapters):
    """
    Compute feature vectors for word and punctuation features
    """
    sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    num_chapters = len(chapters)
    fvs_lexical = np.zeros((len(chapters), 3), np.float64)
    fvs_punct = np.zeros((len(chapters), 3), np.float64)
    for e, ch_text in enumerate(chapters):
        # note: the nltk.word_tokenize includes punctuation
        tokens = n.word_tokenize(ch_text.lower())
        words = word_tokenizer.tokenize(ch_text.lower())
        sentences = sentence_tokenizer.tokenize(ch_text)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                       for s in sentences])

        # average number of words per sentence
        fvs_lexical[e, 0] = words_per_sentence.mean()
        # sentence length variation
        fvs_lexical[e, 1] = words_per_sentence.std()
        # Lexical diversity
        fvs_lexical[e, 2] = len(vocab) / float(len(words))

        # Commas per sentence
        fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
        # Semicolons per sentence
        fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
        # Colons per sentence
        fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))

    # apply whitening to decorrelate the features
    fvs_lexical = whiten(fvs_lexical)
    fvs_punct = whiten(fvs_punct)

    return fvs_lexical, fvs_punct

def SyntacticFeatures(chapters):
    """
    Extract feature vector for part of speech frequencies
    """
    def token_to_pos(ch):
        tokens = n.word_tokenize(ch)
        return [p[1] for p in n.pos_tag(tokens)]

    chapters_pos = [token_to_pos(ch) for ch in chapters]
    pos_list = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS']
    fvs_syntax = np.array([[ch.count(pos) for pos in pos_list]
                           for ch in chapters_pos]).astype(np.float64)

    # normalise by dividing each row by number of tokens in the chapter
    fvs_syntax /= np.c_[np.array([len(ch) for ch in chapters_pos])]

    return fvs_syntax

In [22]:
feature_sets = list(LexicalFeatures(neu[["STATUS"]]))
feature_sets.append(SyntacticFeatures(neu[["STATUS"]]))
feature_sets

/usr/local/lib/python3.4/dist-packages/scipy/cluster/vq.py:148: RuntimeWarning: Some columns have standard deviation zero. The values of these columns will not change.
  RuntimeWarning)


[array([[ 99.58915654,   0.        ,  99.58915654],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        ..., 
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]]), array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 1.,  0.,  0.,  0.,  0.,  0.]])]